# API를 통한 초록 데이터 수집 코드 예제

In [ ]:
# #필요한 경우 아니면 실행시키지 마시오


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
# API를 통한 통합검색에서 학회지의 논문 목록 가져오기
res <- POST(url=search_api,
            accept_json(),
            body = list(
              authKey = my_key,
              searchTerm = '한국문헌정보학회지',
              SearchSort='year',
              sortType='desc',
              startYear = 2017,
              endYear = 2021,
              printRowCnt = 3000
            ),
            encode='form'
            )#POST형식으로 호출을 진행하기 때문에, Body안에 쿼리를 담아서 호출, 검색하고자 하는 학회지를 최근 5년에 대해서 수집
res %>%
  httr::content(as='text', encoding = 'UTF-8') %>%  #tm 패키지를 attach 할때 content()가 실행되지 않는 부분 방지
  fromJSON() -> datafromapi

datafromapi %<>% data.frame()

datalis <-  datafromapi[[1]] %>% data.frame()

datalis %<>% select(lodID,title,pubYear,divFlag)

datalis %>% head()

In [ ]:
# 초록데이터 수집 코드
temp<-NULL


for(i in 1:dim(datalis)[1]){

  res2<- POST(url=view_api,
            accept_json(),
            body = list(
              authKey = my_key,
              lodID = datalis[i,1],
              divFlag = datalis[i,4]
            ),
            encode='form'
            )#POST형식으로 호출을 진행하기 때문에, Body안에 쿼리를 담아서 호출, 
  res2 %>%
  httr::content(as='text', encoding = 'UTF-8') %>% #tm 패키지를 attach 할때 content()가 실행되지 않는 부분 방지
  fromJSON() -> viewfromapi

  viewfromapi %<>% data.frame()

  if(i==1){
    data_abst <- viewfromapi[[1]] %>% data.frame
    data_abst %<>% select(lodID,title,pubYear,abst,keyword)

  }
  else{
    temp <- viewfromapi[[1]] %>% data.frame
    temp %<>% select(lodID,title,pubYear,abst,keyword)

    data_abst %<>% rbind(.,temp)
    temp<-NULL
  }
  

}
#try except문 R에서 구현 실패하여 실제로 수집중 오류가 나면 수동으로 다시 돌려야됨

# 수집한 초록 데이터 전처리 예제

In [ ]:
data1<-fread('C:/#####/데이터사이언스와 R/한국비블리아학회지 abstract.csv')  #비블리아학회지 초록 데이터 가져오기
data2<-fread('C:/#####/데이터사이언스와 R/한국문헌정보학회지 abstract.csv') # 문헌정보학회지 초록 데이터 가져오기
data3<-fread('C:/#####/데이터사이언스와 R/정보관리학회지 abstract.csv')# 정보관리학회지 초록 데이터 가져오기
data_abst<-rbind(data1,data2,data3) # 데이터 결합(python에서는 .append와 같은 역할)
data_abst$abst %<>% str_replace_all("[^[가-힣]]+$", "") #문장에서 영문으로 되어있는 초록이 맨 뒤에 있을 때 전부 대체
data_abst$abst %<>% str_replace_all("^[^[가-힣]]+", "")# 문장에서 영문으로 되어있는 초록이 맨 처음에 있을 때(ex) 한국 비블리아 학회지) 전부 대체
data_abst$abst %<>% stripWhitespace() # 대체 이후 초록 처음과 끝부분에 공백이 존재 할 때 이를 제거
data_abst %<>% na.omit()# API에서 초록 데이터를 가져오지 않아 결측치로 처리될 때 이를 제거
data_abst %<>% filter(abst!='')# 초록 데이터 전처리 이후 아무런 값이 안들어있을 때, 이를 제거 
data_abst$abst %<>% removeNumbers() # 초록 데이터에서 불필요한 숫자 데이터 제거
data_abst$abst %<>% removePunctuation() #초록 데이터에서 특수문자 제거
data_abst$abst<-sapply(data_abst$abst,function(x){paste(extractNoun(x), collapse = " ")}) # 초록데이터에서 명사만 뽑아낸 후, 이를 띄어쓰기로 연결하여 이후 DTM에서 자동으로 토큰화할 수 있게 함
data_abst$abst %>% as.vector() %>% VectorSource() %>% VCorpus() ->corpusd #각 초록 데이터를 문장으로하는 벡터를 만들어 corpus 생성
corpusd <- tm_map(corpusd,tolower) # corpus 내의 모든 영문 단어를 다 소문자로 변환
corpusd <- tm_map(corpusd,removeNumbers) # 미처 제거되지 않은 숫자 제거
corpusd <- tm_map(corpusd,removePunctuation) # 미처 제거되지 않은 특수문자 제거
corpusd <- tm_map(corpusd,removeWords,stwd$V1) # 불용어 사전에서 단어를 가져와 이를 제거
corpusd <- tm_map(corpusd,PlainTextDocument) # corpus안 단어들은 PlainTextDocument의 형태로 전환




# Document Term Matrix(DTM) 생성

In [ ]:
corpusd %>% DocumentTermMatrix(control=list(wordLengths=c(2,Inf))) -> DTMd
#DTM 생성, 잘 분류되지 않고 대부분이 불용어인 한글자인 한글 단어는 제외하여 DTM 생성
#DTM에서 Term만 추출, 이를 바탕으로 gsub 등으로 단어 대체해